In [28]:
import pandas as pd
import numpy as np

import os

import plotly.express as px
import matplotlib.pyplot as plt

In [29]:
zone_lookup = pd.read_csv('uber-trip-data/taxi/taxi-zone-lookup.csv')
zone_lookup.head()

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [30]:
zone_lookup.describe(include='all')

,LocationID,Borough,Zone
count,265.000000,265,265
unique,NaN,7,261
top,NaN,Queens,Governor's Island/Ellis Island/Liberty Island
freq,NaN,69,3
mean,133.000000,NaN,NaN
std,76.643112,NaN,NaN
min,1.000000,NaN,NaN
25%,67.000000,NaN,NaN
50%,133.000000,NaN,NaN
75%,199.000000,NaN,NaN


Aucune valeur manquante. 7 quartiers représentant 265 zones.

In [31]:
folder = "uber-trip-data/"
files = [f for f in os.listdir(folder) if f.endswith('.csv')]

In [32]:
columns_dict = {}

for f in files:
    df = pd.read_csv(os.path.join(folder, f), nrows=0)  # lit uniquement les en-têtes
    columns_dict[f] = df.columns.tolist()

for name, cols in columns_dict.items():
    print(f"\n{name} → {len(cols)} colonnes : {cols}")


uber-raw-data-apr14.csv → 4 colonnes : ['Date/Time', 'Lat', 'Lon', 'Base']

uber-raw-data-aug14.csv → 4 colonnes : ['Date/Time', 'Lat', 'Lon', 'Base']

uber-raw-data-janjune-15.csv → 4 colonnes : ['Dispatching_base_num', 'Pickup_date', 'Affiliated_base_num', 'locationID']

uber-raw-data-jul14.csv → 4 colonnes : ['Date/Time', 'Lat', 'Lon', 'Base']

uber-raw-data-jun14.csv → 4 colonnes : ['Date/Time', 'Lat', 'Lon', 'Base']

uber-raw-data-may14.csv → 4 colonnes : ['Date/Time', 'Lat', 'Lon', 'Base']

uber-raw-data-sep14.csv → 4 colonnes : ['Date/Time', 'Lat', 'Lon', 'Base']


Nous allons travailler sur les données de 2014 qui ont toutes la même structure. Cela permettra d'avoir les longitue et latitude qui ne sont pas présents dans le fichier 2015 

In [33]:
files = ['uber-raw-data-apr14.csv',
'uber-raw-data-aug14.csv',
'uber-raw-data-jul14.csv',
'uber-raw-data-jun14.csv',
'uber-raw-data-may14.csv',
'uber-raw-data-sep14.csv'
]

In [34]:
for f in files:
    df = pd.read_csv(os.path.join(folder, f), nrows=100)  # échantillon
    print(f"\nTypes dans {f} :")
    print(df.dtypes)


Types dans uber-raw-data-apr14.csv :
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

Types dans uber-raw-data-aug14.csv :
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

Types dans uber-raw-data-jul14.csv :
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

Types dans uber-raw-data-jun14.csv :
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

Types dans uber-raw-data-may14.csv :
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

Types dans uber-raw-data-sep14.csv :
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object


Même type de données. On peut concaténer les fichiers dans un même dataframe

In [35]:
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df_2014 = pd.concat(dfs, ignore_index=True)

In [36]:
df_2014.describe(include='all')

,Date/Time,Lat,Lon,Base
count,4534327,4.534327e+06,4.534327e+06,4534327
unique,260093,NaN,NaN,5
top,4/7/2014 20:21:00,NaN,NaN,B02617
freq,97,NaN,NaN,1458853
mean,NaN,4.073926e+01,-7.397302e+01,NaN
std,NaN,3.994991e-02,5.726670e-02,NaN
min,NaN,3.965690e+01,-7.492900e+01,NaN
25%,NaN,4.072110e+01,-7.399650e+01,NaN
50%,NaN,4.074220e+01,-7.398340e+01,NaN
75%,NaN,4.076100e+01,-7.396530e+01,NaN


nous avons dorénavant 4 534 327 lignes

In [37]:
df_2014.isna().sum()

Date/Time    0
Lat          0
Lon          0
Base         0
dtype: int64

Aucune donnée manquante

In [38]:
df_2014.head(5)

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df_2014['Lon'], df_2014['Lat'], 'k+', markersize = 0.2, alpha=0.5)
plt.title('trajets')
plt.grid(True, linestyle='--', alpha=0.8)
plt.xticks(np.arange(-74.4, -73.5, 0.05))
plt.yticks(np.arange(40.5, 41.1, 0.05))
plt.show()

In [40]:
lat_min, lat_max = 40.55, 40.95
lon_min, lon_max = -74.2, -73.7

df_2014_clean = df_2014[
    (df_2014['Lat'] >= lat_min) & (df_2014['Lat'] <= lat_max) &
    (df_2014['Lon'] >= lon_min) & (df_2014['Lon'] <= lon_max)
].copy()

print(f"Nombre de trajets conservés : {len(df_2014_clean)} / {len(df_2014)}")

Nombre de trajets conservés : 4503539 / 4534327


In [ ]:
outliers = df_2014[(df_2014['Lat'] < 40.55) | (df_2014['Lat'] > 40.95) |
              (df_2014['Lon'] < -74.2) | (df_2014['Lon'] > -73.7)]

fig = px.scatter_mapbox(outliers,
                        lat='Lat',
                        lon='Lon',
                        zoom=8,
                        color_discrete_sequence=['red'],
                        title="Outliers géographiques détectés",
                        mapbox_style='carto-positron')

fig.show()


In [42]:
df_2014 = df_2014_clean.copy()
df_2014.shape

(4503539, 4)

Vu le nombre de lignes, nous allons segmenter par jour et heure la date

In [43]:
df_2014['Date/Time'] = pd.to_datetime(df_2014['Date/Time'])
df_2014['day_of_week'] = df_2014['Date/Time'].dt.day_name()
df_2014['hour'] = df_2014['Date/Time'].dt.hour
df_2014.drop(columns=['Date/Time'],axis=1, inplace=True)

In [44]:
df_2014.head(5)

,Lat,Lon,Base,day_of_week,hour
0,40.7690,-73.9549,B02512,Tuesday,0
1,40.7267,-74.0345,B02512,Tuesday,0
2,40.7316,-73.9873,B02512,Tuesday,0
3,40.7588,-73.9776,B02512,Tuesday,0
4,40.7594,-73.9722,B02512,Tuesday,0


Nombre de lignes selon les jours et heures

In [ ]:
day_counts = df_2014.groupby(['Base', 'day_of_week']).size().reset_index(name='count')

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts['day_of_week'] = pd.Categorical(day_counts['day_of_week'],
                                           categories=day_order,
                                           ordered=True)

fig_day = px.bar(day_counts,
                 x='Base',
                 y='count',
                 color='day_of_week',
                 barmode='group',
                 title='Nombre de lignes par Base et par jour de la semaine',
                 labels={'Base': 'Base', 'count': 'Nombre de lignes', 'day_of_week': 'Jour'},
                 color_discrete_sequence=px.colors.qualitative.Vivid)

fig_day.update_layout(template='plotly_white',
                      xaxis_title='Base',
                      yaxis_title='Nombre de lignes')

fig_day.show()

hour_counts = df_2014.groupby(['Base', 'hour']).size().reset_index(name='count')
hour_counts['hour'] = hour_counts['hour'].astype(str)

fig_hour = px.bar(hour_counts,
                  x='Base',
                  y='count',
                  color='hour',
                  barmode='group',
                  title='Nombre de lignes par Base et par heure',
                  labels={'Base': 'Base', 'count': 'Nombre de lignes', 'hour': 'Heure'},
                  color_continuous_scale='Viridis')

fig_hour.update_layout(template='plotly_white',
                       xaxis_title='Base',
                       yaxis_title='Nombre de lignes')

fig_hour.show()


Les mêmes schéma se reproduisent quelque soit la base. 

Voyons maintenant la répartition par jour et par heure

In [ ]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

heatmap_data = df_2014.pivot_table(index='day_of_week', columns='hour', values='Base', aggfunc='count')

heatmap_data = heatmap_data.reindex(day_order)

fig = px.imshow(
    heatmap_data,
    labels=dict(x="Heure", y="Jour de la semaine", color="Nombre de lignes"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    title="Heatmap du nombre de lignes par jour et par heure",
    text_auto=True,
    color_continuous_scale='Viridis',
    aspect="auto"
)

fig.update_layout(template='plotly_white')

fig.show()

Nous pouvons retrouver 2 gros types de journée :
 - en semaine : 
    pic de 7 à 8h et de 15 à 20h
 - du vendredi soir au dimanche : 
    pic du vendredi soir perdure jusqu'au samedi matin 1h puis samedi de 15h à 2h

- Le croisement jour et heure le plus représenté est le jeudi à 17h
- Le jeudi et le vendredi sont les jours avec le plus de courses

Nous allons donc faire une première étude sur ce croisement spécifique

In [47]:
df_jeudi_17 = df_2014[(df_2014['day_of_week'] == 'Thursday') & (df_2014['hour'] == 17)]

In [48]:
df_jeudi_17.describe(include='all')

,Lat,Lon,Base,day_of_week,hour
count,56418.000000,56418.000000,56418,56418,56418.0
unique,NaN,NaN,5,1,NaN
top,NaN,NaN,B02598,Thursday,NaN
freq,NaN,NaN,17798,56418,NaN
mean,40.742834,-73.978259,NaN,NaN,17.0
std,0.030213,0.041251,NaN,NaN,0.0
min,40.574200,-74.199100,NaN,NaN,17.0
25%,40.726100,-73.996300,NaN,NaN,17.0
50%,40.748000,-73.984100,NaN,NaN,17.0
75%,40.760200,-73.972300,NaN,NaN,17.0


Nous retrouvons bien les 56418 lignes de la heatmap

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df_2014['Lon'], df_2014['Lat'], 'k+', markersize = 0.2, alpha=0.5)
plt.title('trajets')
plt.grid(True, linestyle='--', alpha=0.8)
plt.xticks(np.arange(-74.2, -73.7, 0.05))
plt.yticks(np.arange(40.55, 40.95, 0.05))
plt.show()

In [50]:
df_jeudi_17.to_csv('data/jeudi_17.csv')

In [52]:
df_2014.to_csv('data/ens_2014.csv')